In [33]:
import pandas as pd
import datetime
import requests
import urllib
import traceback
from tqdm import tqdm


#The JSON file is too large to upload to Jupyter, I store them in my Dropbox. Use the line bleow to fetch the data.
#df_profile is the profile dataset
#df_games is the game stats dataset

df_profile = pd.read_json('https://www.dropbox.com/s/p8g0ctktl1l0wdv/profiles_1512362725.022629.json?dl=1')
df_games = pd.read_json('https://www.dropbox.com/s/v2dngco51uog1no/games_1512362753.8735218.json?dl=1')



In [39]:
#The dirth date in the dataset is String, use the line bleow to convert the type to datetime.
#Apply this to other date format data if needed
df_profile['birth_date'] = pd.to_datetime(df_profile['birth_date'])

#Selected sub dataset based on birth date
date_after = datetime.date(1977, 1, 1)
df_profile = df_profile[df_profile['birth_date'] > date_after]

#Selected sub dataset based on position
target_position = 'QB'
df_profile = df_profile[df_profile['position'] == target_position]

#reverse selection
#df = df.drop(df[df['birth_date'] > date_after].index)

#reset the index
df_profile = df_profile.reset_index(drop=True)


,birth_date,birth_place,college,current_salary,current_team,death_date,draft_position,draft_round,draft_team,draft_year,height,high_school,hof_induction_year,name,player_id,position,weight
0,1991-05-31,"Thomaston, GA",Baylor,"615,000",New York Jets,None,103.0,4.0,New York Jets,2015.0,6-3,"Midlothian, TX",NaN,Bryce Petty,17509,QB,230.0
1,1988-07-03,None,None,None,None,None,NaN,NaN,None,NaN,6-4,None,NaN,McLeod Bethel-Thompson,1672,QB,230.0
2,1991-09-19,"Gainesville, GA",South Carolina,None,None,None,NaN,NaN,None,NaN,6-0,"Flowery Branch, GA",NaN,Connor Shaw,20023,QB,206.0
3,1982-11-24,"Gilbert, AZ",Harvard,None,Tampa Bay Buccaneers,None,250.0,7.0,St. Louis Rams,2005.0,6-2,"Highland, AZ",NaN,Ryan Fitzpatrick,6975,QB,221.0
4,1990-09-13,"Mobile, AL",Alabama,"690,000",Cincinnati Bengals,None,164.0,5.0,Cincinnati Bengals,2014.0,6-3,"St. Paul's Episcopal, AL",NaN,A.J. McCarron,14490,QB,220.0
5,1985-07-02,"Wyomissing, PA",Michigan,"2,250,000",Jacksonville Jaguars,None,57.0,2.0,Miami Dolphins,2008.0,6-3,"Wilson, PA",NaN,Chad Henne,9636,QB,228.0
6,1979-07-04,"Jacksonville, TX",SMU,None,New York Jets,None,81.0,3.0,Arizona Cardinals,2002.0,6-4,"Jacksonville, TX",NaN,Josh McCown,14580,QB,212.0
7,1980-07-03,"Owosso, MI",Colorado St.,None,None,None,250.0,7.0,Denver Broncos,2004.0,6-2,"San Marcos, CA",NaN,Bradlee Van Pelt,22826,QB,220.0
8,1984-05-30,"Westkake Village, CA",Texas-El Paso,None,None,None,205.0,6.0,Washington Redskins,2007.0,6-5,"Mission Viejo, CA",NaN,Jordan Palmer,17024,QB,232.0
9,1990-04-28,"Scottsdale, AZ",Duke,None,None,None,249.0,7.0,Atlanta Falcons,2013.0,6-3,"Notre Dame Prep, AZ",NaN,Sean Renfree,18464,QB,219.0


In [53]:
#list all the data columns from the game stats
list(df_games)

['age',
 'date',
 'defense_interception_touchdowns',
 'defense_interception_yards',
 'defense_interceptions',
 'defense_sacks',
 'defense_safeties',
 'defense_tackle_assists',
 'defense_tackles',
 'field_goal_attempts',
 'field_goal_makes',
 'game_location',
 'game_number',
 'game_won',
 'kick_return_attempts',
 'kick_return_touchdowns',
 'kick_return_yards',
 'opponent',
 'opponent_score',
 'passing_attempts',
 'passing_completions',
 'passing_interceptions',
 'passing_rating',
 'passing_sacks',
 'passing_sacks_yards_lost',
 'passing_touchdowns',
 'passing_yards',
 'player_id',
 'player_team_score',
 'point_after_attemps',
 'point_after_makes',
 'punt_return_attempts',
 'punt_return_touchdowns',
 'punt_return_yards',
 'punting_attempts',
 'punting_blocked',
 'punting_yards',
 'receiving_receptions',
 'receiving_targets',
 'receiving_touchdowns',
 'receiving_yards',
 'rushing_attempts',
 'rushing_touchdowns',
 'rushing_yards',
 'team',
 'year']

In [57]:
#select the relevant data
#I select all the game stats as an example
datacols = list(df_games.loc[:,'defense_interception_touchdowns':'field_goal_makes']) + list(df_games.loc[:,'kick_return_attempts':'kick_return_yards'])+ list(df_games.loc[:,'passing_attempts':'passing_yards']) + list(df_games.loc[:,'point_after_attemps':'rushing_yards'])

#check the data columes selected
datacols       

['defense_interception_touchdowns',
 'defense_interception_yards',
 'defense_interceptions',
 'defense_sacks',
 'defense_safeties',
 'defense_tackle_assists',
 'defense_tackles',
 'field_goal_attempts',
 'field_goal_makes',
 'kick_return_attempts',
 'kick_return_touchdowns',
 'kick_return_yards',
 'passing_attempts',
 'passing_completions',
 'passing_interceptions',
 'passing_rating',
 'passing_sacks',
 'passing_sacks_yards_lost',
 'passing_touchdowns',
 'passing_yards',
 'point_after_attemps',
 'point_after_makes',
 'punt_return_attempts',
 'punt_return_touchdowns',
 'punt_return_yards',
 'punting_attempts',
 'punting_blocked',
 'punting_yards',
 'receiving_receptions',
 'receiving_targets',
 'receiving_touchdowns',
 'receiving_yards',
 'rushing_attempts',
 'rushing_touchdowns',
 'rushing_yards']

In [71]:
#find the games that our target players played in
df_games = df_games[df_games['player_id'].isin(df_profile['player_id'])]
#drop the rows that player has 0 stats
df_games = df_games[df_games[datacols].sum(axis=1) != 0]
